In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN, QuatE, RotatE, TuckER
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [3]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [4]:
DATA_TYPE = "_transductive_io.tsv"
TRAIN_PATH = "MSCallGraph_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_validation" + DATA_TYPE

In [5]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     testing_path = TEST_PATH,
                     validation_path = VALIDATE_PATH,
                      eager = True
                     )


In [6]:
model_name = 'tuckER_transductive_io'

In [7]:
tracker = ConsoleResultTracker()

In [8]:
loss = NSSALoss() #used by RotatE and NodePiece
embedding_dim = 200

In [9]:
model = TuckER(
        triples_factory=dataset.training,
        random_seed = seed,
        loss = loss,
        embedding_dim = embedding_dim
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

Number of parameters: 11291200
Space occupied: 45164800 bytes


In [10]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory tuckER_transductive_io created successfully!


In [11]:
learning_rate = 1e-2 #1e-3 and 1e-4 stops after 20 iterations and actually does not reduce the error too much
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [12]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        metrics = metrics,
        add_defaults=False
    )

In [13]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.training,
    evaluation_triples_factory = dataset.validation,
    result_tracker = tracker

)



In [14]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.training,
        model=model,
        result_tracker=tracker,
        optimizer=optimizer
)

In [15]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.training,
        num_epochs=num_epochs,
        stopper = stopper
        
    )
training_duration = time.time() - training_start

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 4.493266582489014
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.0006746807517770801
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.000567705816725773
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.0006211932842514267
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.000674680748488754
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0005677058361470699
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0006211932632140815
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0006746807462994187
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.000567705809198503
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0006211932777489608
Metric: validation.head.optimistic.hits_at_1 = 0.00019462826002335538
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 9.7314130011

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 2
Metric: loss = 4.492716550827026
Step: 2
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.0007018587797902758
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.0004990137230912994
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.0006004362514407877
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0007018587202765048
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.000499013694934547
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0006004362367093563
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.000701858734177064
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.0004990137152350733
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0006004362247060687
Metric: validation.head.optimistic.hits_at_1 = 9.731413001167769e-05
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 4.8657065005

...

Step: 35
Metric: loss = 1.1024653911590576
Step: 35
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.024536827664076253
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.00029494145032831403
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.012415884557202285
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.024536827579140663
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0002949414192698896
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.01241588406264782
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.024536826485075464
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.0002949413978890062
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.012415883941482235
Metric: validation.head.optimistic.hits_at_1 = 0.012066952121448035
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 0.006033476060

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 36
Metric: loss = 1.0037274360656738
Step: 36
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.026031244534226262
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.0004355980847978026
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.013233421309512031
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.026031244546175003
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0004355979908723384
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.013233421370387079
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.026031244156794843
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.00043559786983092606
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.01323342101331288
Metric: validation.head.optimistic.hits_at_1 = 0.0126508369015181
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 0.00632541845075

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 37
Metric: loss = 0.907599687576294
Step: 37
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.028405598651189915
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.0006636281756358897
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.0145346134134129
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.02840548008680344
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0006636281032115221
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.014534554444253443
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.028405365542578328
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.0006636280171244934
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.014534496779851408
Metric: validation.head.optimistic.hits_at_1 = 0.0135266640716232
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 0.0067633320358116
M

In [16]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [17]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
           0
0   4.493267
1   4.492717
2   4.492767
3   4.485406
4   4.471889
5   4.454966
6   4.420121
7   4.385030
8   4.322956
9   4.240206
10  4.151405
11  4.034790
12  3.908797
13  3.791850
14  3.659090
15  3.484566
16  3.317972
17  3.195405
18  3.079716
19  2.942002
20  2.818351
21  2.702771
22  2.580211
23  2.468155
24  2.350527
25  2.229536
26  2.103456
27  1.991362
28  1.856860
29  1.734282
30  1.587148
31  1.461664
32  1.347040
33  1.211034
34  1.102465
35  1.003727
36  0.907600


In [18]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.training.mapped_triples,
        additional_filter_triples=[
        dataset.training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/41.5k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.019856   0.019856     0.019856
hits_at_1                     0.011658   0.011658     0.011658
hits_at_3                     0.019438   0.019438     0.019438
hits_at_5                     0.024617   0.024617     0.024617
hits_at_10                    0.033650   0.033650     0.033650
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.000726   0.000726     0.000726
hits_at_1                     0.000145   0.000145     0.000145
hits_at_3                     0.000409   0.000409     0.000409
hits_at_5                     0.000626   0.000626     0.000626
hits_at_10                    0.001204   0.001204     0.001204
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.010291   0.010291     0.010291
hits_at_1                     0.005901   0.005901     0.005901
hits_at_3                     0.009924  

In [19]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/10.3k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.031874   0.031874     0.031874
hits_at_1                     0.019366   0.019366     0.019366
hits_at_3                     0.031043   0.031043     0.031043
hits_at_5                     0.040969   0.040969     0.040969
hits_at_10                    0.054107   0.054107     0.054107
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.001271   0.001271     0.001271
hits_at_1                     0.000292   0.000292     0.000292
hits_at_3                     0.000681   0.000681     0.000681
hits_at_5                     0.001362   0.001362     0.001362
hits_at_10                    0.002822   0.002822     0.002822
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.016572   0.016572     0.016572
hits_at_1                     0.009829   0.009829     0.009829
hits_at_3                     0.015862  

In [20]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples,
            dataset.validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/13.1k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.030713   0.030713     0.030713
hits_at_1                     0.017587   0.017587     0.017587
hits_at_3                     0.030356   0.030356     0.030356
hits_at_5                     0.041061   0.041061     0.041061
hits_at_10                    0.054290   0.054290     0.054290
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.001546   0.001546     0.001546
hits_at_1                     0.000459   0.000459     0.000459
hits_at_3                     0.001300   0.001300     0.001300
hits_at_5                     0.001912   0.001912     0.001912
hits_at_10                    0.002676   0.002676     0.002676
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.016129   0.016129     0.016129
hits_at_1                     0.009023   0.009023     0.009023
hits_at_3                     0.015828  

In [21]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [22]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [23]:
infodict['loss'] = NSSALoss
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience

In [24]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                   embedding_dim   
17                   learning_rate   
18                       optimizer   
19                      num_epochs   
20                        patience   

                                                value  
0                                              cuda:0  
1                                            451648

In [25]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)